In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [1]:
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [2]:
print(len(text))

1115394


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
chars = sorted(list(set(text)))
vocab_size = len(chars)

In [5]:
print(f"[{''.join(chars)}]")
print(vocab_size)

[
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz]
65


# Simplest baseline

## Simple Encoder and Decoder

In [6]:
stoi = {ch:i for i, ch in enumerate(chars)}
itos = {i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

In [14]:
print(encode('hello'))
print(decode([46, 43, 50, 50, 53]))

[46, 43, 50, 50, 53]
hello


In [16]:
# import tiktoken
# enc = tiktoken.get_encoding('gpt2')
# print(enc.n_vocab)

50257


In [19]:
# print(enc.encode('hello world'))
# print(enc.decode([31373, 995]))

[31373, 995]
hello world


In [7]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)

torch.Size([1115394]) torch.int64


## Train valid split

In [8]:
n = int(0.9 * len(data))
train_data, valid_data = data[:n], data[n:]
print(train_data.shape, valid_data.shape)

torch.Size([1003854]) torch.Size([111540])


=> Train on small chunks

In [9]:
block_size = 32
train_data[:block_size]

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[: t+1]
    target = y[t]
    print(context, target)

tensor([18]) tensor(47)
tensor([18, 47]) tensor(56)
tensor([18, 47, 56]) tensor(57)
tensor([18, 47, 56, 57]) tensor(58)
tensor([18, 47, 56, 57, 58]) tensor(1)
tensor([18, 47, 56, 57, 58,  1]) tensor(15)
tensor([18, 47, 56, 57, 58,  1, 15]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47]) tensor(58)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58]) tensor(47)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47]) tensor(64)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64]) tensor(43)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43]) tensor(52)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52]) tensor(10)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10]) tensor(0)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0]) tensor(14)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14]) tensor(43)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43]) tensor(44)
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64

=> Multiple batches of chunk size

In [11]:
batch_size = 4
block_size = 32

def get_batch(split):
    data = train_data if split == 'train' else valid_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

x, y = get_batch('train')
print(x.shape, y.shape)
print("X = ", x)
print("Y = ", y)

torch.Size([4, 32]) torch.Size([4, 32])
X =  tensor([[43,  1, 39, 52, 42,  1, 56, 43, 60, 43, 52, 45, 43,  0, 32, 47, 50, 50,
          1, 58, 46, 53, 59,  1, 58, 46, 43,  1, 50, 47, 43,  7],
        [58, 46, 63,  1, 45, 56, 43, 63, 46, 53, 59, 52, 42, 57,  1, 39, 56, 43,
          1, 39, 57,  1, 57, 61, 47, 44, 58,  0, 13, 57,  1, 40],
        [ 0, 18, 47, 56, 57, 58,  1, 31, 43, 52, 39, 58, 53, 56, 10,  0, 25, 63,
          1, 45, 43, 52, 43, 56, 39, 50,  1, 41, 39, 56, 43, 57],
        [56, 57, 58,  1, 39, 44, 44, 43, 41, 58, 47, 53, 52, 10,  1, 46, 47, 57,
          1, 59, 52, 48, 59, 57, 58,  1, 59, 52, 49, 47, 52, 42]])
Y =  tensor([[ 1, 39, 52, 42,  1, 56, 43, 60, 43, 52, 45, 43,  0, 32, 47, 50, 50,  1,
         58, 46, 53, 59,  1, 58, 46, 43,  1, 50, 47, 43,  7, 45],
        [46, 63,  1, 45, 56, 43, 63, 46, 53, 59, 52, 42, 57,  1, 39, 56, 43,  1,
         39, 57,  1, 57, 61, 47, 44, 58,  0, 13, 57,  1, 40, 56],
        [18, 47, 56, 57, 58,  1, 31, 43, 52, 39, 58, 53, 56, 10,  0

## Model

In [12]:
import torch.nn as nn
from torch.nn import functional as F

class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, idx, targets=None):
        logits = self.token_embedding(idx) # (batch_size, block_size, vocab_size)

        if targets is None:
            loss = None
        else:
            logits = logits.view(-1, vocab_size) # (batch_size * block_size, vocab_size)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # (batch_size, vocab_size)
            probs = F.softmax(logits, dim=-1)
            idx_nxt = torch.multinomial(probs, num_samples=1) # sample from the distribution (B, 1)
            idx = torch.cat((idx, idx_nxt), dim=1)
        return idx
    
model = BigramLanguageModel(vocab_size)
out, loss = model(x, y)
print(loss, out.shape)

print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


tensor(4.6030, grad_fn=<NllLossBackward0>) torch.Size([128, 65])

DZ3xgRCnS;?ruRCf$Q&O$A-HMQwT
X
rF';?ncPJHDpjyysz
uh;qccuBvdN
IVuO3hA&:ODF 'FhUUARWFnFFn3TB-lLbK;:Bw$


## Training

In [13]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BigramLanguageModel(vocab_size).to(device)

In [14]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

batch_size = 32
eval_iter = 100
for epoch in range(10000):
    model.train()
    x, y = get_batch('train')
    x, y = x.to(device), y.to(device)
    _, loss = model(x, y)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if epoch % 1000 == 0:
        model.eval()
        loss = torch.zeros(eval_iter, device=device)
        for k in range(eval_iter):
            x, y = get_batch('valid')
            x, y = x.to(device), y.to(device)
            logits, loss[k] = model(x, y)

        loss = loss.mean()
        print(f"Epoch {epoch}, Loss = {loss.item()}")
        model.train()

print(loss.item())

Epoch 0, Loss = 4.728024959564209
Epoch 1000, Loss = 3.6272764205932617
Epoch 2000, Loss = 3.019394874572754
Epoch 3000, Loss = 2.722482204437256
Epoch 4000, Loss = 2.5922181606292725
Epoch 5000, Loss = 2.542281150817871
Epoch 6000, Loss = 2.510268211364746
Epoch 7000, Loss = 2.499406337738037
Epoch 8000, Loss = 2.491530656814575
Epoch 9000, Loss = 2.4872887134552
2.430380344390869


In [15]:
print(decode(model.generate(idx = torch.zeros((1, 1), dtype=torch.long).to(device), max_new_tokens=300)[0].tolist()))


We te y t pespeloe asey thed be.
Aner-t-
Gofist, knerve theamy t dentshe pNGSve, s y r
CHar she ha ty woun, ome t, nou sched apray tht m dfon ssouet hen, saioulfar agl

Thagus he be, th d om can rshace har t.
De t ath INCHENII'M3d ame om smy EOused o I'dlscu nd benoay uce Thrat ICon, belima now, pe.


## Mathematical trick in self-attention

In [16]:
B, T, C = 4, 32, 65
x = torch.randn(B, T, C)
x.shape

torch.Size([4, 32, 65])

=> Only take postition in previous step, must not look into future

In [17]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3)) # => Magic: take only previous tokens => mask in self-attention mechanism
a = a / torch.sum(a, 1, keepdim=True) # Distribution over previous tokens
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [18]:
xbow = torch.zeros((B, T, C))
for b in range(B):
    for t in range(T):
        xprev = x[b, :t+1] # (t,C)
        xbow[b, t] = torch.mean(xprev, 0)

In [19]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)

xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [20]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
print(wei)
wei = F.softmax(wei, dim=-1)
print(f"After softmax \n{wei}")

xbow3 = wei @ x
torch.allclose(xbow, xbow3)

tensor([[0., -inf, -inf,  ..., -inf, -inf, -inf],
        [0., 0., -inf,  ..., -inf, -inf, -inf],
        [0., 0., 0.,  ..., -inf, -inf, -inf],
        ...,
        [0., 0., 0.,  ..., 0., -inf, -inf],
        [0., 0., 0.,  ..., 0., 0., -inf],
        [0., 0., 0.,  ..., 0., 0., 0.]])
After softmax 
tensor([[1.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.0333, 0.0333, 0.0333,  ..., 0.0333, 0.0000, 0.0000],
        [0.0323, 0.0323, 0.0323,  ..., 0.0323, 0.0323, 0.0000],
        [0.0312, 0.0312, 0.0312,  ..., 0.0312, 0.0312, 0.0312]])


False

In [21]:
torch.allclose(xbow2, xbow3)

True

In [26]:
import torch.nn as nn
from torch.nn import functional as F

n_embed = 32
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding = nn.Embedding(vocab_size, vocab_size)
        self.postition_embedding_table = nn.Embedding(block_size, n_embed)
        self.lm_head = nn.Linear(n_embed, vocab_size)
    
    def forward(self, idx, targets=None):
        B, T = idx.shape
        tok_emb = self.token_embedding(idx) # (batch_size, block_size, vocab_size)
        pos_emb = self.postition_embedding_table(torch.arange(T, device=device)) # (block_size, n_embed)
        x = tok_emb + pos_emb
        logits = self.lm_head(x) # (batch_size, block_size, vocab_size)

        if targets is None:
            loss = None
        else:
            logits = logits.view(-1, vocab_size) # (batch_size * block_size, vocab_size)
            targets = targets.view(-1)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:, -1, :] # (batch_size, vocab_size)
            probs = F.softmax(logits, dim=-1)
            idx_nxt = torch.multinomial(probs, num_samples=1) # sample from the distribution (B, 1)
            idx = torch.cat((idx, idx_nxt), dim=1)
        return idx
    
    
model = BigramLanguageModel(vocab_size).to(device)

## Self-Attention

In [27]:
# version 4: self-attention!
torch.manual_seed(1337)
B, T, C = 4, 32, 65
x = torch.randn(B, T, C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

# print(wei)
v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 32, 16])

In [28]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [29]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 2000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B,T,C = x.shape
        k = self.key(x)   # (B,T,C)
        q = self.query(x) # (B,T,C)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5 # (B, T, C) @ (B, C, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,C)
        out = wei @ v # (B, T, T) @ (B, T, C) -> (B, T, C)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(n_embd, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel()
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


0.209729 M parameters
step 0: train loss 4.4116, val loss 4.4022
step 100: train loss 2.6568, val loss 2.6670
step 200: train loss 2.5090, val loss 2.5058
step 300: train loss 2.4194, val loss 2.4334
step 400: train loss 2.3497, val loss 2.3558
step 500: train loss 2.2968, val loss 2.3132
step 600: train loss 2.2408, val loss 2.2497
step 700: train loss 2.2052, val loss 2.2187
step 800: train loss 2.1639, val loss 2.1868
step 900: train loss 2.1237, val loss 2.1499
step 1000: train loss 2.1031, val loss 2.1304
step 1100: train loss 2.0693, val loss 2.1176
step 1200: train loss 2.0385, val loss 2.0794
step 1300: train loss 2.0246, val loss 2.0648
step 1400: train loss 1.9928, val loss 2.0366
step 1500: train loss 1.9697, val loss 2.0304
step 1600: train loss 1.9605, val loss 2.0450
step 1700: train loss 1.9404, val loss 2.0133
step 1800: train loss 1.9092, val loss 1.9954
step 1900: train loss 1.9112, val loss 1.9893
step 1999: train loss 1.8835, val loss 1.9935

And they bridchwiand is